In [1]:
%pip install yfinance

Note: you may need to restart the kernel to use updated packages.


In [1]:
import yfinance as yf
import pandas as pd
import numpy as np

# def simulate_macd_strategy(ticker, year, initial_budget):
ticker = 'MSFT'
year = '2024'
initial_budget = 1000

# --- 1. Pobranie danych ---
start_date = f"{year}-01-01"
end_date = f"{year}-12-31"

df = yf.download(ticker, start=start_date, end=end_date)

if df.empty:
    raise ValueError("Brak danych – sprawdź ticker lub zakres dat.")

df.to_csv(f"{ticker}.csv")


/tmp/ipykernel_45943/4129849565.py:14: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=start_date, end=end_date)
[*********************100%***********************]  1 of 1 completed


In [2]:
df.head().T

,Date,2024-01-02,2024-01-03,2024-01-04,2024-01-05,2024-01-08
Price,Ticker,,,,,
Close,MSFT,3.654216e+02,3.651556e+02,3.625347e+02,3.623475e+02,3.691855e+02
High,MSFT,3.703777e+02,3.677765e+02,3.676189e+02,3.665942e+02,3.696880e+02
Low,MSFT,3.613818e+02,3.630963e+02,3.617760e+02,3.611158e+02,3.635889e+02
Open,MSFT,3.683676e+02,3.635890e+02,3.652246e+02,3.635496e+02,3.638746e+02
Volume,MSFT,2.525860e+07,2.308350e+07,2.090150e+07,2.100460e+07,2.313400e+07


In [3]:

# --- 2. Oblicz MACD ---
df["EMA12"] = df["Close"].ewm(span=12, adjust=False).mean()
df["EMA26"] = df["Close"].ewm(span=26, adjust=False).mean()
df["MACD"] = df["EMA12"] - df["EMA26"]
df["Signal"] = df["MACD"].ewm(span=9, adjust=False).mean()


In [4]:

# sygnały
df["BuySignal"] = ((df["MACD"] > df["Signal"]) & (df["MACD"].shift(1) <= df["Signal"].shift(1))).astype(int)
df["SellSignal"] = ((df["MACD"] < df["Signal"]) & (df["MACD"].shift(1) >= df["Signal"].shift(1))).astype(int)


In [5]:
df.head().T

,Date,2024-01-02,2024-01-03,2024-01-04,2024-01-05,2024-01-08
Price,Ticker,,,,,
Close,MSFT,3.654216e+02,3.651556e+02,3.625347e+02,3.623475e+02,3.691855e+02
High,MSFT,3.703777e+02,3.677765e+02,3.676189e+02,3.665942e+02,3.696880e+02
Low,MSFT,3.613818e+02,3.630963e+02,3.617760e+02,3.611158e+02,3.635889e+02
Open,MSFT,3.683676e+02,3.635890e+02,3.652246e+02,3.635496e+02,3.638746e+02
Volume,MSFT,2.525860e+07,2.308350e+07,2.090150e+07,2.100460e+07,2.313400e+07
EMA12,,3.654216e+02,3.653807e+02,3.649428e+02,3.645435e+02,3.652577e+02
EMA26,,3.654216e+02,3.654019e+02,3.651895e+02,3.649790e+02,3.652906e+02
MACD,,0.000000e+00,-2.121624e-02,-2.466748e-01,-4.354379e-01,-3.288695e-02
Signal,,0.000000e+00,-4.243248e-03,-5.272956e-02,-1.292712e-01,-1.099944e-01


In [6]:

# --- 3. Symulacja portfela ---
cash = initial_budget
shares = 0
last_buy_cash = None  # zapamiętanie kapitału przed zakupem (do obliczenia zysku)

trades = []

for i in range(len(df) - 1):
    today = df.iloc[i]
    tomorrow = df.iloc[i + 1]

    # --- KUPNO ---
    if today["BuySignal",''] == 1 and cash > 0:
        price = tomorrow["Open",ticker]
        shares = cash / price
        last_buy_cash = cash  # zapamiętujemy ile mieliśmy przed kupnem
        cash = 0

        trades.append({
            "date": tomorrow.name,
            "type": "BUY",
            "price": price,
            "shares": shares,
            "cash_after": cash,
            "profit_abs": None,
            "profit_pct": None
        })

    # --- SPRZEDAŻ ---
    elif today["SellSignal",''] == 1 and shares > 0:
        price = tomorrow["Open",ticker]
        sell_value = shares * price
        cash = sell_value

        # obliczenie zysku od momentu zakupu
        profit_abs = cash - last_buy_cash if last_buy_cash is not None else None
        profit_pct = (profit_abs / last_buy_cash * 100) if last_buy_cash else None

        trades.append({
            "date": tomorrow.name,
            "type": "SELL",
            "price": price,
            "shares": 0,
            "cash_after": cash,
            "profit_abs": profit_abs,
            "profit_pct": profit_pct
        })

        shares = 0
        last_buy_cash = None

# --- 4. Wartość końcowa ---
final_price = df.iloc[-1]["Close"]
final_value = cash + shares * final_price

summary = {
    "ticker": ticker,
    "year": year,
    "initial_budget": initial_budget,
    "final_value": final_value,
    "profit": final_value - initial_budget,
    "return_%": (final_value / initial_budget - 1) * 100,
    "trades": trades
}

    # return summary




: 

: 

In [8]:
print(summary)

{'ticker': 'MSFT', 'year': '2024', 'initial_budget': 1000, 'final_value': Ticker
MSFT    1086.168147
Name: 2024-12-30 00:00:00, dtype: float64, 'profit': Ticker
MSFT    86.168147
Name: 2024-12-30 00:00:00, dtype: float64, 'return_%': Ticker
MSFT    8.616815
Name: 2024-12-30 00:00:00, dtype: float64, 'trades': [{'date': Timestamp('2024-01-09 00:00:00'), 'type': 'BUY', 'price': np.float64(366.5449108276047), 'shares': np.float64(2.728178650038126), 'cash_after': 0, 'profit_abs': None, 'profit_pct': None}, {'date': Timestamp('2024-02-14 00:00:00'), 'type': 'SELL', 'price': np.float64(402.81865792132635), 'shares': 0, 'cash_after': np.float64(1098.961262377974), 'profit_abs': np.float64(98.96126237797398), 'profit_pct': np.float64(9.896126237797398)}, {'date': Timestamp('2024-03-15 00:00:00'), 'type': 'BUY', 'price': np.float64(413.8942801973478), 'shares': np.float64(2.6551738329265655), 'cash_after': 0, 'profit_abs': None, 'profit_pct': None}, {'date': Timestamp('2024-04-01 00:00:00'), '

In [9]:
# ------------------------------------
# PRZYKŁADOWE URUCHOMIENIE
# ------------------------------------
# result = simulate_macd_strategy("MSFT", 2024, 1000)
result = summary

print("=== PODSUMOWANIE STRATEGII MACD ===")
print(f"Ticker: {result['ticker']}")
print(f"Rok: {result['year']}")
print(f"Kapitał początkowy: {result['initial_budget']}$")
print(f"Wartość końcowa portfela: {result['final_value']:.2f}$")
print(f"Zysk/Strata: {result['profit']:.2f}$")
print(f"Zwrot: {result['return_%']:.2f}%")

print("\n=== TRANSACJE ===")
for t in result["trades"]:
    date = t["date"].date()
    ttype = t["type"]
    price = t["price"]
    shares = t["shares"]
    cash_after = t["cash_after"]

    print(f"{date} | {ttype} | cena: {price:.2f} | akcje: {shares:.4f} | gotówka: {cash_after:.2f}")

    if t["type"] == "SELL":
        print(f"   → Wynik transakcji: {t['profit_abs']:.2f}$ ({t['profit_pct']:.2f}%)")


=== PODSUMOWANIE STRATEGII MACD ===
Ticker: MSFT
Rok: 2024
Kapitał początkowy: 1000$


TypeError: unsupported format string passed to Series.__format__